In [2]:
path = 'train_data_sila/train_data.csv'

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv(path)
data['full_text'] = data['Тема'] + ' ' + data['Описание']

In [5]:
data

,index,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер,full_text
0,0,Не работает блок питания,В течении недели перестал внезапно работать бл...,Ноутбук,Блок питания,C223100360,Не работает блок питания В течении недели пере...
1,1,Не включается SILA LLC HK2-3404.,Добрый день!_x000D_\nПодскажите пожалуйста что...,Ноутбук,Материнская плата,C223094534,Не включается SILA LLC HK2-3404. Добрый день!_...
2,2,Сервер СР2-5422 // Системные ошибки // D251110041,Добрый вечер! На сервере появились системные о...,Сервер,Материнская плата,D251110041,Сервер СР2-5422 // Системные ошибки // D251110...
3,3,Повреждение матрицы,Прошу произвести диагностику и сориентировать ...,Ноутбук,Матрица,C223014328,Повреждение матрицы Прошу произвести диагности...
4,4,sila HK2-1404 c223014125 asset 7101087,Добрый день!\nПрошу взять ноутбук sila HK2-140...,Ноутбук,Материнская плата,C223014125,sila HK2-1404 c223014125 asset 7101087 Добрый ...
...,...,...,...,...,...,...,...
189,189,НК2-1404 // Не работает Wi-FI (Кашира)// C2231...,На устройстве не работает WiFi\n,Ноутбук,Wi-fi модуль,C223101219,НК2-1404 // Не работает Wi-FI (Кашира)// C2231...
190,190,Не работает блок питания (зарядка),Не работает блок питания (зарядка),Ноутбук,Блок питания,С223093223,Не работает блок питания (зарядка) Не работает...
191,191,Ноутбук НК2-1404 //Не работает клавиша F1 или ...,"Добрый день,\nпосле ремонта ноутбука перестала...",Ноутбук,Клавиатура,С223010731,Ноутбук НК2-1404 //Не работает клавиша F1 или ...
192,192,Отказ блока питания,Добрый день! \n У нас перестал работать зарядн...,Ноутбук,Блок питания,C222091750,Отказ блока питания Добрый день! \n У нас пере...


In [6]:
data['Серийный номер'].unique()

array(['C223100360', 'C223094534', 'D251110041', 'C223014328',
       'C223014125', 'C223010123', 'C223091001', 'C223012430',
       'D252030021', 'c223012961', 'C223100312', 'C223090725',
       'C223102255', 'CKM01212505744', 'С111120102', 'D252110012',
       'С112040045', 'C222090950', 'C223090096', 'C223010035',
       'C223102298', 'C223091126', 'C223010076', 'C223013256',
       'С223090320', 'C223013523', 'C223014561', 'C223010345, С223092735',
       'D263120004', 'D114030017(FTKY614)', 'С111080061', 'C223010446',
       'D252030009',
       'C223011170, C223102198, C223011889, C223101201, C223100299, C223011531',
       'D251110015', 'C222090246', 'D254020003', 'C223091623',
       'C223012998 C223101389  C223102298   C223100876  C223100050',
       'D263090003', 'C223013250', 'C223011660', 'C233041042',
       'C223011088', 'С111080030', 'C223091624', 'D251110011',
       'C223014954', 'С223010395', 'C223012095', 'С223091324',
       'С223091153', 'C223090893', 'C223090123',

In [5]:
data.sample(1)

,index,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер,full_text
61,61,СИЛА НК2-1404 // Отвалился разъем type c // C2...,Отвалился разъем type c на зарядном устройстве,Ноутбук,Блок питания,C223012413,СИЛА НК2-1404 // Отвалился разъем type c // C2...


## zero-shot-classify-SSTuning-XLM-R

In [36]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, string, random

tokenizer = AutoTokenizer.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")
model = AutoModelForSequenceClassification.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")

sample = data.sample(1)
text = sample['Описание'].values[0]
true_label = sample['Тип оборудования'].values[0]
list_label = data['Тип оборудования'].unique().tolist()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
list_ABC = [x for x in string.ascii_uppercase]

def check_text(model, text, list_label, shuffle=False): 
    list_label = [x+'.' if x[-1] != '.' else x for x in list_label]
    list_label_new = list_label + [tokenizer.pad_token]* (20 - len(list_label))
    if shuffle: 
        random.shuffle(list_label_new)
    s_option = ' '.join(['('+list_ABC[i]+') '+list_label_new[i] for i in range(len(list_label_new))])
    text = f'{s_option} {tokenizer.sep_token} {text}'

    model.to(device).eval()
    encoding = tokenizer([text],truncation=True, max_length=512,return_tensors='pt')
    item = {key: val.to(device) for key, val in encoding.items()}
    logits = model(**item).logits
    
    logits = logits if shuffle else logits[:,0:len(list_label)]
    probs = torch.nn.functional.softmax(logits, dim = -1).tolist()
    predictions = torch.argmax(logits, dim=-1).item() 
    probabilities = [round(x,5) for x in probs[0]]

    return list_label_new[predictions]

print(text)
print(check_text(model, text, list_label), true_label)

Аварийно перезагрузился сервер, в логе BMC появилось совпадающее по времени событие:_x000D_
20	 Critical	2024-05-31 17:38:27 (UTC+3)_x000D_
Intel ME has encountered UMA operation error. Details: UMA Read/Write timeout. Timeout occurred during copying data from/to UMA._x000D_
_x000D_
Просьба продиагностировать, возможно ли использовать данный сервер в продуктивном контуре ?
Сервер. Сервер


In [7]:
from tqdm.notebook import tqdm

In [8]:
list_label = data['Тип оборудования'].unique().tolist()
pred, ans = [], []
for row in tqdm(data.iterrows()):
    text = row[1]['Описание']
    true_label = row[1]['Тип оборудования']
    pred_label = check_text(model, text, list_label)
    pred.append(pred_label)
    ans.append(true_label)
pred = [x.replace('.', '') for x in pred]

0it [00:00, ?it/s]

In [9]:
from sklearn.metrics import classification_report
print(classification_report(pred, ans))

              precision    recall  f1-score   support

     Ноутбук       0.85      0.99      0.92       132
         СХД       0.91      0.30      0.45        33
      Сервер       0.72      0.72      0.72        29

    accuracy                           0.84       194
   macro avg       0.83      0.67      0.70       194
weighted avg       0.84      0.84      0.81       194



In [37]:
list_label = data['Точка отказа'].unique().tolist()
pred, ans = [], []
for row in tqdm(data.iterrows()):
    text = row[1]['full_text']
    true_label = row[1]['Точка отказа']
    pred_label = check_text(model, text, list_label)
    pred.append(pred_label)
    ans.append(true_label)
pred = [x.replace('.', '') for x in pred]

0it [00:00, ?it/s]

In [38]:
from sklearn.metrics import classification_report
print(classification_report(pred, ans))

                         precision    recall  f1-score   support

                   Jack       0.00      0.00      0.00         1
             SFP модуль       1.00      1.00      1.00         1
          Wi-fi антенна       0.50      0.67      0.57         3
           Wi-fi модуль       0.25      0.33      0.29         3
            Аккумулятор       1.00      0.50      0.67         2
           Блок питания       0.55      0.64      0.59        33
             Вентилятор       0.29      0.40      0.33         5
               Динамики       1.00      0.14      0.25         7
                   Диск       0.73      0.73      0.73        11
                  Диск        0.20      0.50      0.29         2
                 Камера       0.00      0.00      0.00         3
             Клавиатура       0.29      0.67      0.40         3
           Консультация       0.46      0.10      0.16        62
                 Корпус       0.00      0.00      0.00         3
      Материнская плата 

C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

## MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7

In [10]:
# !pip install tf_keras

In [39]:
from transformers import pipeline
import numpy as np
from tqdm.notebook import tqdm
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device='cuda:0')
sequence_to_classify = "Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output['labels'][np.argmax(output['scores'])])

politics


In [40]:
np.argmax(output['scores'])

0

In [13]:
output

{'sequence': 'Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU',
 'labels': ['politics', 'economy', 'environment', 'entertainment'],
 'scores': [0.9658798575401306,
  0.022846847772598267,
  0.007333965040743351,
  0.003939332906156778]}

In [14]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device='cuda:0')

list_label = data['Тип оборудования'].unique().tolist()
pred, ans = [], []
for row in tqdm(data.iterrows()):
    text = row[1]['Описание']
    true_label = row[1]['Тип оборудования']
    output = classifier(text, list_label, multi_label=False)
    pred_label = output['labels'][np.argmax(output['scores'])]
    pred.append(pred_label)
    ans.append(true_label)

0it [00:00, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [15]:
from sklearn.metrics import classification_report
print(classification_report(pred, ans))

              precision    recall  f1-score   support

     Ноутбук       0.60      0.99      0.74        93
         СХД       0.82      0.21      0.33        43
      Сервер       0.79      0.40      0.53        58

    accuracy                           0.64       194
   macro avg       0.74      0.53      0.54       194
weighted avg       0.70      0.64      0.59       194



In [41]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device='cuda:0')

list_label = data['Точка отказа'].unique().tolist()
pred, ans = [], []
for row in tqdm(data.iterrows()):
    text = row[1]['Описание']
    true_label = row[1]['Точка отказа']
    output = classifier(text, list_label, multi_label=False)
    pred_label = output['labels'][np.argmax(output['scores'])]
    pred.append(pred_label)
    ans.append(true_label)

0it [00:00, ?it/s]

In [42]:
from sklearn.metrics import classification_report
print(classification_report(pred, ans))

                         precision    recall  f1-score   support

                   Jack       0.00      0.00      0.00         1
             SFP модуль       0.00      0.00      0.00         0
          Wi-fi антенна       0.25      1.00      0.40         1
           Wi-fi модуль       0.25      0.50      0.33         2
            Аккумулятор       0.00      0.00      0.00        13
           Блок питания       0.24      0.82      0.37        11
             Вентилятор       0.43      0.43      0.43         7
               Динамики       1.00      0.06      0.11        17
                   Диск       0.91      0.91      0.91        11
                  Диск        0.00      0.00      0.00         2
                 Камера       0.00      0.00      0.00         2
             Клавиатура       0.57      0.80      0.67         5
           Консультация       0.31      0.11      0.16        38
                 Корпус       0.33      0.06      0.10        17
      Материнская плата 

C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

## Cosine similarity with RuBert

In [44]:
# Load model directly
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").cuda()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
from transformers import AutoTokenizer, AutoModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

text = data['Описание'].values.tolist()
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {key: value.to(device) for key, value in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_state = outputs.last_hidden_state
cls_features = last_hidden_state[:, 0, :]
cls_features = cls_features.cpu()

In [46]:
list_label = data['Тип оборудования'].unique().tolist()
class_input = tokenizer(list_label, return_tensors="pt", padding=True, truncation=True, max_length=512)
class_input = {key: value.to(device) for key, value in class_input.items()}

In [47]:
with torch.no_grad():
    class_outputs = model(**class_input)

last_hidden_state = class_outputs.last_hidden_state
class_features = last_hidden_state[:, 0, :]
class_features = class_features.cpu()

In [48]:
class_features.shape

torch.Size([3, 768])

In [49]:
cls_features.shape

torch.Size([194, 768])

In [50]:
import torch

# Normalize the class_features and cls_features to unit vectors
class_features_norm = torch.nn.functional.normalize(class_features, p=2, dim=1)
cls_features_norm = torch.nn.functional.normalize(cls_features, p=2, dim=1)

# Compute the cosine similarity matrix
cosine_sim_matrix = torch.matmul(cls_features_norm, class_features_norm.T)

# Get the best class for each row in cls_features
best_class_indices = torch.argmax(cosine_sim_matrix, dim=1)

# Convert the indices to class labels if needed
best_class_labels = best_class_indices.tolist()

# If you want the actual cosine similarity values for the best class:
best_cosine_sim_values = torch.max(cosine_sim_matrix, dim=1).values

# Now best_class_labels contains the best class for each row in cls_features
# and best_cosine_sim_values contains the corresponding cosine similarity values

In [51]:
dict_ = {
    0:list_label[0],
    1:list_label[1],
    2:list_label[2],
}

In [52]:
pred = [dict_[x] for x in best_class_labels]
ans = data['Тип оборудования'].tolist()

In [53]:
from sklearn.metrics import classification_report
print(classification_report(pred, ans))

              precision    recall  f1-score   support

     Ноутбук       0.81      0.79      0.80       158
         СХД       0.09      0.03      0.05        33
      Сервер       0.00      0.00      0.00         3

    accuracy                           0.65       194
   macro avg       0.30      0.27      0.28       194
weighted avg       0.68      0.65      0.66       194



In [62]:
list_label = data['Точка отказа'].unique().tolist()
class_input = tokenizer(list_label, return_tensors="pt", padding=True, truncation=True, max_length=512)
class_input = {key: value.to(device) for key, value in class_input.items()}

with torch.no_grad():
    class_outputs = model(**class_input)

last_hidden_state = class_outputs.last_hidden_state
class_features = last_hidden_state[:, 0, :]
class_features = class_features.cpu()

import torch

# Normalize the class_features and cls_features to unit vectors
class_features_norm = torch.nn.functional.normalize(class_features, p=2, dim=1)
cls_features_norm = torch.nn.functional.normalize(cls_features, p=2, dim=1)

# Compute the cosine similarity matrix
cosine_sim_matrix = torch.matmul(cls_features_norm, class_features_norm.T)

# Get the best class for each row in cls_features
best_class_indices = torch.argmax(cosine_sim_matrix, dim=1)

# Convert the indices to class labels if needed
best_class_labels = best_class_indices.tolist()

# If you want the actual cosine similarity values for the best class:
best_cosine_sim_values = torch.max(cosine_sim_matrix, dim=1).values

# Now best_class_labels contains the best class for each row in cls_features
# and best_cosine_sim_values contains the corresponding cosine similarity values

dict_ = dict(zip(range(len(list_label)), list_label))

pred = [dict_[x] for x in best_class_labels]
ans = data['Точка отказа'].tolist()

In [63]:
from sklearn.metrics import classification_report
print(classification_report(pred, ans))

                         precision    recall  f1-score   support

                   Jack       0.00      0.00      0.00         0
             SFP модуль       0.00      0.00      0.00        17
          Wi-fi антенна       0.50      0.04      0.07        55
           Wi-fi модуль       0.75      0.08      0.14        39
            Аккумулятор       0.00      0.00      0.00        14
           Блок питания       0.11      0.40      0.17        10
             Вентилятор       0.00      0.00      0.00         0
               Динамики       0.00      0.00      0.00         3
                   Диск       0.00      0.00      0.00         0
                  Диск        0.00      0.00      0.00         0
                 Камера       0.00      0.00      0.00         1
             Клавиатура       0.00      0.00      0.00         0
           Консультация       0.00      0.00      0.00         4
                 Корпус       0.00      0.00      0.00         0
      Материнская плата 

C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Илья\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

In [56]:
dict_ = dict(zip(range(len(list_label)), list_label))

In [57]:
dict_

{0: 'Блок питания',
 1: 'Материнская плата',
 2: 'Матрица',
 3: 'Вентилятор',
 4: 'Сервер',
 5: 'Wi-fi модуль',
 6: 'Диск',
 7: 'Консультация',
 8: 'SFP модуль',
 9: 'Диск ',
 10: 'Оперативная память',
 11: 'Программное обеспечение',
 12: 'Клавиатура',
 13: 'Корпус',
 14: 'Аккумулятор',
 15: 'Камера',
 16: 'Wi-fi антенна',
 17: 'Динамики',
 18: 'Jack'}

In [28]:
data['Серийный номер'].unique()

array(['C223100360', 'C223094534', 'D251110041', 'C223014328',
       'C223014125', 'C223010123', 'C223091001', 'C223012430',
       'D252030021', 'c223012961', 'C223100312', 'C223090725',
       'C223102255', 'CKM01212505744', 'С111120102', 'D252110012',
       'С112040045', 'C222090950', 'C223090096', 'C223010035',
       'C223102298', 'C223091126', 'C223010076', 'C223013256',
       'С223090320', 'C223013523', 'C223014561', 'C223010345, С223092735',
       'D263120004', 'D114030017(FTKY614)', 'С111080061', 'C223010446',
       'D252030009',
       'C223011170, C223102198, C223011889, C223101201, C223100299, C223011531',
       'D251110015', 'C222090246', 'D254020003', 'C223091623',
       'C223012998 C223101389  C223102298   C223100876  C223100050',
       'D263090003', 'C223013250', 'C223011660', 'C233041042',
       'C223011088', 'С111080030', 'C223091624', 'D251110011',
       'C223014954', 'С223010395', 'C223012095', 'С223091324',
       'С223091153', 'C223090893', 'C223090123',

In [32]:
data.iloc[-7]['Серийный номер']

'C223091216 C223091307 C223091468 C223091751 C223090099 C223090635 C223091364 C223091547 C223091736 C223091275 C223091861 C223090091 C223090302 C223090269 C223090214'